# Demonstration of statistical tracking in BioPharma optimisation

The basic [optimisation demo](Optimisation_demo.ipynb) showed how to find and visualise the best solution from among a population of parameter settings. This notebook demonstrates how to track and retrieve information about the evolution of the population as the optimisation runs. In particular, it addresses:
- how to specify which parameters to track
- what kind of information is available
- how to visualise these results.

We start by creating and running an optimiser, as in the basic demo. 
While adding a variable parameter, we can tell the optimiser to track its evolution during the optimisation by compiling appropriate statistics. There are two possible sets of statistics, for numerical and categorical (discrete) values, and we must specify which kind we want to use. In this example, we choose to track .... All other variables are not tracked by default.

In [ ]:
import biopharma as bp

facility = bp.Facility(data_path='data')

# Define the steps needed to create our single product
from biopharma.process_steps import (
    
)
steps = [
    
]
product = bp.Product(facility, steps)

In [ ]:
from biopharma import optimisation as opt

optimiser = opt.Optimiser(facility)

# Specify the variables to optimise.
# Note the 'track=opt.Tracking.discrete' here, which says to track the singleUse parameter as a discrete variable,
# counting the number of individuals within the population that have each value for this variable.
optimiser.add_variable(gen=opt.gen.Binary(), component=opt.sel.step('test_step'), item='binary_param',
                       track=opt.Tracking.discrete)
# Numerical tracking is specified for the next variable, meaning statistics on its distribution across the
# population will be computed.
optimiser.add_variable(gen=opt.gen.RangeGenerator(0, 10),
                       component=opt.sel.step('test_step'), item='int_param', track=opt.Tracking.numerical)

# Specify the objective
optimiser.add_objective(component=opt.sel.product(0), item='cogs', minimise=True)

# Run the optimisation
optimiser.run()

## Reviewing statistics

Beyond the data on the best solution, the optimiser can provide additional information on the solutions it considered at each generation, as well as how these evolved during the optimisation. This is accessible through its logbook, from which we can select the particular information we are interested in.

We can start by inspecting how the fitness of the population changed across the generations. The logbook has information on the minimum, maximum, average and standard deviation of the fitness at each generation:

In [ ]:
import pandas as pd

col_names = ["min", "max", "avg", "std"]
fit_df = pd.DataFrame({col: optimiser.logbook.chapters["fit"].select(col) for col in col_names},
                      index=optimiser.logbook.select("gen"))
fit_df.index.name = "Generation"
fit_df

The result for each generation and statistic is a list, because in general we could be optimising multiple objectives at the same time. Each element of this list corresponds to the fitness for a particular objective. Here, the lists have length 1, as we have provided a single objective.

In [ ]:
import numpy as np
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
output_notebook()

gens = optimiser.logbook.select("gen")
fit_data = optimiser.logbook.chapters["fit"]
# The "fit" chapter has information on the fitness; we need to select the data related
# to the objective we are interested in (in this case there is only one).
# Using NumPy arrays in case data contains infinite values or NaNs, or Bokeh will complain
fit_min = np.array([value[0] for value in fit_data.select("min")])
fit_max = np.array([value[0] for value in fit_data.select("max")])
fit_avg = np.array([value[0] for value in fit_data.select("avg")])

# Create plot
p = figure(title='Population fitness')
p.line(gens, fit_min, line_color='blue', legend='min')
p.line(gens, fit_max, line_color='red', legend='max')
p.line(gens, fit_avg, line_color='green', legend='avg')
p.xaxis.axis_label = 'Generation'
p.yaxis.axis_label = 'Fitness'
p.title.text_font_size = '11pt'
show(p)

Note that sometimes the fitness value is infinite, in which case it will not be shown in the plot!

We can also plot the standard deviation of the fitness, to show whether the fitness values converge as the optimisation continues.

In [ ]:
import numpy as np

fit_std = np.array([value[0] for value in fit_data.select("std")])

p = figure(title='Population fitness (mean +/- 1 std)')
p.line(gens, fit_avg)
p.line(gens, fit_avg + fit_std, line_dash='dashed')
p.line(gens, fit_avg - fit_std, line_dash='dashed')
p.xaxis.axis_label = 'Generation'
p.yaxis.axis_label = 'Fitness'
p.title.text_font_size = '11pt'
show(p)

In addition to the fitness, we can retrieve information on any of the parameters we asked the optimiser to track. The full list of tracked quantities is available in the logbook's chapters:

In [ ]:
for parameter in optimiser.logbook.chapters:
    print(parameter)

For numerical parameters (specified with `track=opt.Tracking.numerical` when adding them to the optimiser), the same measures (minimum, maximum, average and standard deviation) are available:

In [ ]:
gens = optimiser.logbook.select("gen")
diam_data = optimiser.logbook.chapters["test_step[int_param]"]
# Strip the units off the result, to plot only the magnitudes
diam_min = [value.magnitude for value in diam_data.select("min")]
diam_max = [value.magnitude for value in diam_data.select("max")]
diam_avg = [value.magnitude for value in diam_data.select("avg")]
unit = optimiser.logbook.chapters["test_step[int_param]"].select("min")[0].units

p = figure(title='Evolution of parameter in test step')
p.line(gens, diam_min, line_color='blue', legend='min')
p.line(gens, diam_max, line_color='red', legend='max')
p.line(gens, diam_avg, line_color='green', legend='avg')
p.xaxis.axis_label = 'Generation'
p.yaxis.axis_label = 'Parameter ({})'.format(unit)
p.title.text_font_size = '11pt'
show(p)

For categorical parameters (`track=opt.Tracking.discrete`), instead, we can see how often each value appears. This is stored as the `count` measure.

In [ ]:
# Apparently, the high-level bokeh.charts is no longer maintained
# and has been replaced by HoloViews. We could look at including that,
# but we can still do something simpler with Bokeh.
from bokeh.models import ColumnDataSource
from bokeh.models.tickers import FixedTicker
from bokeh.palettes import viridis
from bokeh.core.properties import value

# For technical reasons, the result for each generation is a list,
# and the actual data is contained in its first and only element.
use_data = [result['count'][0] for result in optimiser.logbook.chapters["test_step[binary_param]"]]
all_options = set()
for result in use_data:
    all_options.update(result.keys())
all_options = list(all_options)

# Put the results in a connvenient format for plotting
compiled_data = {str(option): [result.get(option, 0) for result in use_data] for option in all_options}
all_options_str = list(compiled_data.keys())
xlabel = 'Generation'
compiled_data[xlabel] = gens
source = ColumnDataSource(compiled_data)
colours = viridis(len(all_options_str))

p = figure()
p.vbar_stack(all_options_str, x=xlabel, source=source,
             width=0.7, color=colours, legend=[value(opt) for opt in all_options_str])

# Make the plot look a little nicer
p.xaxis.axis_label = xlabel
p.xaxis.ticker = FixedTicker(ticks=gens)
p.yaxis.axis_label = 'Number of individuals'
p.title.text = 'Distribution of characteristic "test_step[binary_param]"'
p.title.text_font_size = '11pt'
show(p)